<a href="https://colab.research.google.com/github/cheonghf/ML-P4-03/blob/main/Project_SourceCode_P4_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

dataset_1 = 'https://raw.githubusercontent.com/cheonghf/ML-P4-03/refs/heads/main/bank_transactions_data_2.csv'

ds1 = pd.read_csv(dataset_1, index_col = 0)

print(ds1)


              AccountID  TransactionAmount      TransactionDate  \
TransactionID                                                     
TX000001        AC00128              14.09  2023-04-11 16:29:14   
TX000002        AC00455             376.24  2023-06-27 16:44:19   
TX000003        AC00019             126.29  2023-07-10 18:16:08   
TX000004        AC00070             184.50  2023-05-05 16:32:11   
TX000005        AC00411              13.45  2023-10-16 17:51:24   
...                 ...                ...                  ...   
TX002508        AC00297             856.21  2023-04-26 17:09:36   
TX002509        AC00322             251.54  2023-03-22 17:36:48   
TX002510        AC00095              28.63  2023-08-21 17:08:50   
TX002511        AC00118             185.97  2023-02-24 16:24:46   
TX002512        AC00009             243.08  2023-02-14 16:21:23   

              TransactionType          Location DeviceID      IP Address  \
TransactionID                                       